In [1]:
import numpy as np

In [2]:
import scipy.sparse

In [3]:
import pickle

In [ ]:
import xgboost as xgb

# 从文件或者 xgboost 生成的二进制缓存中读取数据

In [5]:
dtrain = xgb.DMatrix('demo/data/agaricus.txt.train')
dtest = xgb.DMatrix('demo/data/agaricus.txt.test')

# 用 dict 保存参数

In [7]:
param = {
    'max_depth': 2,
    'eta': 1,
    'silent': 1,
    'objective': 'binary:logistic'
}

# 设置 validation set 

In [8]:
watchlist = [(dtest, 'eval'), (dtrain, 'train')]

In [9]:
num_round = 2

In [10]:
bst = xgb.train(param, dtrain, num_round, watchlist)

[0]	eval-error:0.042831	train-error:0.046522
[1]	eval-error:0.021726	train-error:0.022263


# 预测

In [11]:
preds = bst.predict(dtest)

In [12]:
labels = dtest.get_label()

In [13]:
labels

array([0., 1., 0., ..., 1., 0., 1.], dtype=float32)

In [14]:
preds

array([0.28583017, 0.9239239 , 0.28583017, ..., 0.9239239 , 0.05169873,
       0.9239239 ], dtype=float32)

In [15]:
preds > 0.5

array([False,  True, False, ...,  True, False,  True])

In [23]:
pred_labels = np.copy(preds)

# 选择阈值进行二分类

In [30]:
threshold = 0.5

In [31]:
pred_labels[preds > threshold] = 1

In [32]:
pred_labels[preds <= threshold] = 0

In [33]:
pred_labels

array([0., 1., 0., ..., 1., 0., 1.], dtype=float32)

In [34]:
print(sum(pred_labels == labels) / labels.shape)

[0.97827436]


In [37]:
sum(np.array([True, False, True, False]))

2

In [38]:
bst.save_model('test.model')

# dump model: 这个不是很理解，是说可以把模型和特征一一对应起来输出吗？

In [49]:
bst.dump_model('dump.raw.txt')

In [50]:
bst.dump_model('dump.nice.txt', 'demo/data/featmap.txt')

In [51]:
cat dump.nice.txt

booster[0]:
0:[odor=pungent] yes=2,no=1
	1:[stalk-root=cup] yes=4,no=3
		3:leaf=1.71218
		4:leaf=-1.70044
	2:[spore-print-color=orange] yes=6,no=5
		5:leaf=-1.94071
		6:leaf=1.85965
booster[1]:
0:[stalk-root=missing] yes=2,no=1
	1:[odor=pungent] yes=4,no=3
		3:leaf=0.784718
		4:leaf=-0.96853
	2:leaf=-6.23624


In [52]:
cat dump.raw.txt

booster[0]:
0:[f29<-9.53674e-07] yes=1,no=2,missing=1
	1:[f56<-9.53674e-07] yes=3,no=4,missing=3
		3:leaf=1.71218
		4:leaf=-1.70044
	2:[f109<-9.53674e-07] yes=5,no=6,missing=5
		5:leaf=-1.94071
		6:leaf=1.85965
booster[1]:
0:[f60<-9.53674e-07] yes=1,no=2,missing=1
	1:[f29<-9.53674e-07] yes=3,no=4,missing=3
		3:leaf=0.784718
		4:leaf=-0.96853
	2:leaf=-6.23624


看结果，好像是用特征名称代替了数字？